In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
!pip install transformers
!pip3 install torch torchvision
!pip install sentence-transformers

In [30]:
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from transformers import BertModel, BertForMaskedLM
from transformers import AutoModel
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
from pprint import pprint

In [20]:
def concaten(token_embeddings):
  # concat
  token_vecs_cat = []

  for token in token_embeddings:
      
      cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4], token[-5], token[-6]), dim=0)
      
      token_vecs_cat.append(cat_vec)

  # print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))
  
  return token_vecs_cat

In [21]:
def summ(token_embeddings):
  # sum
  token_vecs_sum = []

  for token in token_embeddings:
    
      sum_vec = torch.sum(token[-6:], dim=0)
      
      token_vecs_sum.append(sum_vec)

  # print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

  return token_vecs_sum

In [78]:
def funcyo(text, text2, cat, token):

  tokenized_text = tokenizer.tokenize(text, padding=True, truncation=True, return_tensors='pt')
  tokenized_text.append('[SEP]')
  tokenized_text.insert(0, '[CLS]')

  tokenized_text2 = tokenizer.tokenize(text2, padding=True, truncation=True, return_tensors='pt')
  tokenized_text = tokenized_text + tokenized_text2

  nulls = 1 + tokenized_text.index('[SEP]')

  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [0] * nulls + [1] * (len(tokenized_text) - nulls)


  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])


  with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)

  hidden_states = outputs[2]

  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  token_embeddings = token_embeddings.permute(1,0,2)
  #token_embeddings.size()

  if cat:
    token_vecs_m = concaten(token_embeddings)
  else:
    token_vecs_m = summ(token_embeddings)

  token_vecs = hidden_states[-2][0]
  sentence_embedding = torch.mean(token_vecs, dim=0)

  diff_bank = 0
  a = b = c = d = -1
  if token not in tokenized_text or token not in tokenized_text2:
    indices = [i for i, x in enumerate(tokenized_text) if x in token]
    for i in range(len(indices)):
      if i+1 < len(indices) and indices[i+1]-indices[i]==1:
        if a == -1:
          a = i
          b = i+1
        else:
          c = i
          d = i+1
    diff_bank1 = 1 - cosine(token_vecs_m[indices[a]], token_vecs_m[indices[c]])
    diff_bank2 = 1 - cosine(token_vecs_m[indices[b]], token_vecs_m[indices[d]])
    # diff_bank3 = 1 - cosine(token_vecs_m[indices[0]], token_vecs_m[indices[1]])
    # diff_bank = (diff_bank1 + diff_bank2 + diff_bank3) / 3
    diff_bank = (diff_bank1 + diff_bank2) / 2

  else:
    indices = [i for i, x in enumerate(tokenized_text) if x == token]
    if len(indices) > 1:
      diff_bank = 1 - cosine(token_vecs_m[indices[0]], token_vecs_m[indices[1]])
    #print('Vector similarity:  %.2f' % diff_bank)

  return diff_bank

In [18]:
sheet = "Stavki anotirani rocno"
wb = gc.open(sheet)
rows = wb.sheet1.get_all_values();

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EMBEDDIA/sloberta")
#tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")
#tokenizer = BertTokenizer.from_pretrained("EMBEDDIA/crosloengual-bert")

model = BertModel.from_pretrained("EMBEDDIA/sloberta", output_hidden_states = True)
#model = BertModel.from_pretrained('bert-base-multilingual-uncased', output_hidden_states = True)
#model = BertModel.from_pretrained('EMBEDDIA/crosloengual-bert', output_hidden_states = True)

model.eval()

In [82]:
wb2 = gc.open('output').sheet1
count = 1;

for row in rows:

  if row[0] != "":
    token = "▁" + row[0]

    text1 = row[2]
    text2 = row[3]
    cat = True # change to false to use sum instead
    diff = funcyo(text1, text2, cat, token)

    cells = wb2.range('A'+str(count)+':K'+str(count))

    cells[0].value = str(diff)
    #cells[1].value = row[0]
    #cells[2].value = 
    #cells[3].value = row[2]
    #cells[4].value = row[3]

    wb2.update_cells(cells)

    count += 1